In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
%matplotlib
import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,10)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly 
import plotly.express as ex
import seaborn as sns
from plotly.io import templates
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder


sns.set(style= 'whitegrid', color_codes=True)
sns.set_theme(context='notebook',style='darkgrid',
              palette='deep',font='sans-serif',font_scale=1,color_codes=True,rc=None)
%matplotlib inline

In [39]:
Expenditure = pd.read_csv("../input/medical-insurance-payout/expenses.csv")

In [40]:
Expenditure.head(4)

In [41]:
#Check the null value
Expenditure.isnull().sum()

In [42]:
df=Expenditure

In [43]:
#Change categorical value into int
df['smoker']=Expenditure.smoker.map(dict(yes=1, no=0))
df['sex']=Expenditure.sex.map(dict(male=1, female=0))

In [44]:
plt.figure(figsize=(12,10))
sns.heatmap(df.corr(), vmin=-1, cmap="YlGnBu_r", annot=True)

In [45]:
Children=Expenditure.children.value_counts().rename_axis('NumberofChildren').reset_index(name='count')

In [46]:
plt.figure(figsize=(10,10))
colors = ['#FFF8DC', '#9BCD9B', '#66CDAA', '#20B2AA',"#009ACD","#104E8B"]

plt.pie(Children["count"].tolist(), labels=Children["NumberofChildren"].tolist(), labeldistance=1.15, wedgeprops = { 'linewidth' : 1, 'edgecolor' : 'white' }, colors=colors,
       autopct='%1.1f%%',shadow=True);
plt.title("Number of Children")

In [47]:
region=Expenditure.region.value_counts().rename_axis('region').reset_index(name='count')

In [48]:
plt.figure(figsize=(10,10))
colors = ['#FFF8DC', '#9BCD9B', '#66CDAA', '#20B2AA']

plt.pie(region["count"].tolist(), labels=region["region"].tolist(), labeldistance=1.15, wedgeprops = { 'linewidth' : 1, 'edgecolor' : 'white' }, colors=colors,
       autopct='%1.1f%%', shadow=True);
plt.title("Region Distribution")

In [49]:
plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
sns.distplot(Expenditure.age, kde = True, color='#0000FF', kde_kws={'color':'black'})
plt.show()

In [50]:
plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
sns.distplot(Expenditure.bmi, kde = True, color='#0000FF', kde_kws={'color':'black'})
plt.show()

In [51]:
plt.figure(figsize=(8,5))
sns.countplot(x='smoker', data = Expenditure, palette='GnBu', saturation=0.8)

In [52]:
plt.figure(figsize=(8,5))
sns.countplot(x='sex', data = Expenditure, palette='GnBu', saturation=0.8)

In [53]:
plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
sns.distplot(Expenditure.charges, kde = True, color='#0000FF', kde_kws={'color':'black'})
plt.show()

# Model Building

In [54]:
#convert "region" in to dummies columns
dummies = pd.get_dummies(df.region)

In [55]:
df1 = pd.concat([df.drop('region',axis='columns'),dummies],axis='columns')

In [56]:
x=df1.drop(columns="charges")
y=df1.charges

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [58]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()
LR.fit(x_train,y_train)
LR.score(x_test,y_test)

In [59]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), x, y, cv=cv)

In [60]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(x,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(x,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score'])

find_best_model_using_gridsearchcv(x,y)

In [61]:
df1

In [62]:
def predict_charges(region,age,sex,bmi,children,smoker):    
    reg_index  = np.where(x.columns==region)[0][0]

    a = np.zeros(len(x.columns))
    a[0] = age
    a[1] = sex
    a[2] = bmi
    a[3] = children
    a[4] = smoker
    if reg_index >= 0:
        a[reg_index] = 1

    return LR.predict([a])[0]

In [63]:
predict_charges('southwest',23, 0, 27, 0, 1)

In [64]:
predict_charges('southwest',23, 0, 27, 0, 0)

In [65]:
import pickle
with open('MedicalExpenditure','wb') as f:
    pickle.dump(LR,f)

In [66]:
import json
columns = {
    'data_columns' : [col.lower() for col in x.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))